# SOLO models: debugging
This notebook is to examine operation of models based upon a single data file, in order to evaluate how well the models work etc.

In [ ]:
# file handling
from glob import glob
import os

# data + modelling
import math
import numpy as np
import pandas as pd
import pymc3 as pm

# plotting
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt

# set up plotting preferences
plt.style.use('seaborn-darkgrid')

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

%matplotlib inline
# %matplotlib notebook

In [ ]:
# autoreload imported modules. Convenient while I'm developing the code.
%load_ext autoreload
%autoreload 2

## Import the models and other stuff we need

In [ ]:
from models_solo import *
from df_data import *

# Data import
For more info on this, see the **Data preparation** notebook.

In [ ]:
files = glob('data/non_parametric/*.txt')

## Step 1: create experiment level metadata

In [ ]:
import os as os
from collections import namedtuple

def parse_filename(fname):
    """Extract experiment meta data from a filename. Return as a named tuple 
    where the fieldname will become the column header in the experiment meta data
    table."""
    path, file = os.path.split(fname)
    initials = file.split('-')[0]
    domain = file.split('-')[1]
    Metadata = namedtuple('Metadata', ['filename', 'initials', 'domain'])
    return Metadata(filename = fname, initials=initials, domain=domain)

expt_data = build_metadata(files, parse_filename)
expt_data.head()

## Step 2: Import raw behavioural data

In [ ]:
raw_data = import_raw_data(expt_data['filename'])
raw_data.head()

## Select one data file to examine
We are doing things a little differently here. The focus of this notebook is debugging, so we are going to focus on individual raw data files and the corresponding parameter estimates.

In [ ]:
data = raw_data[raw_data['file_index'] == 0]
data.head()

# Parameter estimation: inspect results from multiple models
Create a simple custom visualisation function that displays diagnostic plots inside this notebook.

In [ ]:
def plot_stuff(model, data):
    """Plot discount rates and diagnostic stuff. Plot target is inline in the notebook, not do disc."""
    fig, ax = plt.subplots(figsize=(12, 8))
    discount_function_plotter(ax, [model], data, 'temp', 'misc_participant', export=False)
    plt.show()
    
    trace_df = pm.trace_to_dataframe(model.trace, varnames=model.df_params)
    sns.pairplot(trace_df)
    pm.traceplot(model.trace, varnames=model.df_params)
    pm.autocorrplot(model.trace, varnames=model.df_params)
    pm.forestplot(model.trace, varnames=model.df_params)

## Coinflip model

In [ ]:
c = Coinflip()
c.sample_posterior(data, nsamples=2000)
plot_stuff(c, data)

## Exponential

In [ ]:
e = Exponential()
e.sample_posterior(data, nsamples=2000, target_accept=.95, tune=2000)

plot_stuff(e, data)

## Hyperbolic model

In [ ]:
h = Hyperbolic()
h.sample_posterior(data, nsamples=2000, target_accept=.95, tune=2000)

plot_stuff(h, data)

## Hyperbolic with magnitude effect

In [ ]:
hma = HyperbolicMagnitudeEffect()
hma.sample_posterior(data, nsamples=2000, target_accept=.95, tune=2000)

plot_stuff(hma, data)

## HyperboloidA

In [ ]:
ha = HyperboloidA()
ha.sample_posterior(data, nsamples=2000, target_accept=.95, tune=2000)

plot_stuff(ha, data)

## HyperboloidB

In [ ]:
hb = HyperboloidB()
hb.sample_posterior(data, nsamples=2000, target_accept=.95, tune=2000)

plot_stuff(hb, data)

## Hyperbolic log

In [ ]:
hl = HyperbolicLog()
hl.sample_posterior(data, nsamples=200, target_accept=.95, tune=500)

plot_stuff(hl, data)

## BetaDelta

In [ ]:
bd = BetaDelta()
bd.sample_posterior(data, nsamples=2000, target_accept=.95, tune=2000)

plot_stuff(bd, data)

## DoubleExponential

In [ ]:
data = get_data_df_for_id_num(alldata, 0)

de = DoubleExponential()
de.sample_posterior(data, nsamples=2000, target_accept=.95, tune=2000)

plot_stuff(de, data)

## ConstantSensitivity

In [ ]:
cs = ConstantSensitivity()
cs.sample_posterior(data, nsamples=2000, target_accept=.95, tune=2000)

plot_stuff(cs, data)

## Exponential Power

In [ ]:
ep = ExponentialPower()
ep.sample_posterior(data, nsamples=2000, target_accept=.95, tune=2000)

plot_stuff(ep, data)

## Exponential Log

In [ ]:
el = ExponentialLog()
el.sample_posterior(data, nsamples=2000, tune=2000)


plot_stuff(el, data)

# Below are Heuristic models
As in models not based upon discounted utility. These are in-progress.

# EXPERIMENTAL: Trade-off model by Scholten & Read (2010)

In [ ]:
to = TradeOff()
to.sample_posterior(data, nsamples=2000, target_accept=.95, tune=1000)
plot_stuff(to, data)

# Experimental: ITCH model

In [ ]:
itch = ITCH()
itch.sample_posterior(data, nsamples=2000, target_accept=.95, tune=2000)
plot_stuff(itch, data)

# DRIFT model, Read et al (2013)

**WARNING:** This model breaks when `reward A > reward B`

In [ ]:
drift = DRIFT()
drift.sample_posterior(data, nsamples=2000, target_accept=.95, tune=2000)
plot_stuff(drift, data)